# Temporal-Difference Learning

Chapter 6 follows the use of Temporal-Difference (TD) learning - a method which combines the approaches of the Dynamic Programming and Monte Carlo approaches to learning. With TD learning - we update our understanding of the values as we go along, rather than at the end of an episode (as we did with Monte Carlo approaches). We are still learning through independant trials, so we don't need to fully understand the environment as we do using DP approaches, but we learn as we go through the episode.

This method in practice converges faster than Monte Carlo (although there is no formal proof that this is always the case), and is also suitable for a wider range of problems, as we no longer need to wait until the end of an episode to learn. We will demonstrate the TD approach to value prediction and the control problem using the Windy Gridworld example.

## Windy Gridworld

Windy Gridworld is an environment our agent can act in. The world is an NxM grid, with a start & goal position for the player, and the player can move in a cardinal direction. Windy gridword adds wind in, which applies to some columns and pushes the player up a square. This provides a simple environment to implement the basic concepts of TD learning

In [283]:
from enum import Enum
import numpy as np

class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __add__(self, other):
        return Position(self.x + other.x, self.y + other.y)
    
    def __hash__(self):
        return hash(repr(self))
    
    def __repr__(self):
        return '({0}, {1})'.format(self.x, self.y)

class Action(Enum):
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3 
    
    def position(self):
        if self == Action.UP:
            return Position(0, -1)
        elif self == Action.DOWN:
            return Position(0, 1)
        elif self == Action.LEFT:
            return Position(-1, 0)
        elif self == Action.RIGHT:
            return Position(1, 0)
        else:
            return None

class WindyGridworld:
    
    def __init__(self, width: int, height: int, start: Position, goal: Position, windy_cols):
        if len(windy_cols) != width:
            raise Exception('Number of windy cols must match the width of the grid')
        
        self.width = width
        self.height = height
        self.start = start
        self.goal = goal
        self.windy_cols = windy_cols
        
        self.current_position = start
        
    def act(self, action: Action):
        wind = self.windy_cols[self.current_position.x]
        self.current_position += action.position()
        
        # Apply the wind
        self.current_position.y -= wind
        
        # Bound the player to the walls of the Grid
        if (self.current_position.x < 0):
            self.current_position.x = 0
        elif (self.current_position.x >= self.width):
            self.current_position.x = self.width - 1
        
        if (self.current_position.y < 0):
            self.current_position.y = 0
        elif (self.current_position.y >= self.height):
            self.current_position.y = self.height - 1
            
        return self.current_position
    
    def reward(self):
        if self.current_position == self.goal:
            return 0
        else:
            return -1
    
    def terminal(self):
        return self.current_position == self.goal
    
    def __repr__(self):
        world = np.zeros((self.height, self.width))
        world[self.start.y, self.start.x] = 1
        world[self.goal.y, self.goal.x] = 2
        world[self.current_position.y, self.current_position.x] = 3
        
        return str(world) + '\n\n ' + str(self.windy_cols)
    
    

## Sarsa

The Sarsa algorithm uses TD(0) to determine the value, and then a policy (in this case we are using epsilon greedy) to determine the action to take based off the value estime. TD(0) updates the value estimates mid-episode (as opposed to Monte Carlo which does it at the end of the episode) so it learns every action.

In [284]:
# Define our policy. We will use an epsilon-greedy policy
def epsilon_greedy(state, q_values, epsilon):
    if random.random() < epsilon:
        return random.choice(list(Action))
    
    
    current_max = None
    current_actions = []

    for a in Action:
        val = q_values[(state, a)]
        if current_max == None or val > current_max:
            current_max = val
            current_actions = [a]
        elif val == current_max:
            current_actions.append(a)
    
    return random.choice(current_actions)


In [300]:
import random

HEIGHT = 7
WIDTH = 10

Q = {(Position(x, y), action): 0 for action in Action for x in range(0, WIDTH) for y in range(0, HEIGHT)}
START = Position(0, 3)
GOAL = Position(7, 3)
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

EPISODES = 1000
ALPHA = 0.5
EPSILON = 0.1
GAMMA = 1

policy = epsilon_greedy

timesteps = 0

for e in range(0, EPISODES):
    environment = WindyGridworld(WIDTH, HEIGHT, START, GOAL, WIND)
    s = START
    a = policy(s, Q, EPSILON)

    while not environment.terminal():
        s_prime = environment.act(a)
        r = environment.reward()
        a_prime = policy(s_prime, Q, EPSILON)
        Q[(s, a)] = Q[(s, a)] + ALPHA * (r + GAMMA * Q[(s_prime, a_prime)] - Q[(s, a)])
        
        s = s_prime
        a = a_prime
        
        timesteps += 1
        
print("{} episodes complete in {} steps".format(EPISODES, timesteps))
    

1000 episodes complete in 24320 steps


In [307]:
VALIDATION_STEPS = 1
total_steps = 0

for i in range(0, VALIDATION_STEPS):
    validation_env = WindyGridworld(WIDTH, HEIGHT, START, GOAL, WIND)
    s = START
    actions = []
    states = [s]
    while not validation_env.terminal():
        a = policy(s, Q, 0)
        actions.append(a)
        s = validation_env.act(a)
        states.append(s)
        
    total_steps += len(actions)
    
print("Average steps per episode: {}".format(total_steps / VALIDATION_STEPS))
    

Average steps per episode: 15.0


In [304]:
def action_to_letter(action):
    if action == Action.LEFT:
        return 'L'
    elif action == Action.RIGHT:
        return 'R'
    elif action == Action.UP:
        return 'U'
    elif action == Action.DOWN:
        return 'D'
    else:
        return '?'
    
t = [[action_to_letter(epsilon_greedy(Position(x, y), Q, 0)) for x in range(0, WIDTH)] for y in range(0, HEIGHT)]

In [309]:
t = [['X' if Position(x, y) in states else ' ' for x in range(0, WIDTH)] for y in range(0, HEIGHT)]

In [310]:
t

[[' ', ' ', ' ', ' ', ' ', ' ', 'X', 'X', 'X', 'X'],
 [' ', ' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', 'X'],
 [' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', ' ', 'X'],
 ['X', 'X', 'X', 'X', ' ', ' ', ' ', 'X', ' ', 'X'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'X', 'X'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]